<a href="https://colab.research.google.com/github/Averybipossum/Averybipossum/blob/main/NSEE_Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fazer um pip install dbfread

In [ ]:
# imports necessários
import pandas as pd
from dbfread import DBF
import numpy as np

# Mudar a path qunado for terstar o sistema <<<<<<<<<
table = DBF("C:/Users/rutra/Downloads/pacigeral_jun24.dbf", encoding='latin-1')

# Transformando em dataframe panda
dados = pd.DataFrame(iter(table))

# 1. Selecionar pacientes com Topografia de pulmão (TOPOGRUP = C34);
topo_pulmao = dados.where(dados['TOPOGRUP'].isin(['C34']))
#print(topo_pulmao.head())

# 2. Selecionar pacientes com estado de Residência de São Paulo (UFRESID = SP);
ufresid = dados.where(dados['UFRESID'].isin(['SP']))
#print(ufresid.head())

# 3. Selecionar pacientes com Base do Diagn ́ostico com Confirmação Microscópica (BASEDIAG = 3);
diag_micro = dados.where(dados['BASEDIAG'].isin([3]))
#print(diag_micro.head())

# 4. Retirar categorias 0, X e Y da coluna ECGRUP;
dados['ECGRUP'] = dados['ECGRUP'].where(~dados['ECGRUP'].isin([0, 'X', 'Y']))
novoEcgrup = dados['ECGRUP']
# print(novoEcgrup.head())

# 5. Retirar pacientes que fizeram Hormonioterapia e TMO (HORMONIO = 1 e TMO = 1);
dados['HORMONIO'] = dados['HORMONIO'].where(~dados['HORMONIO'] == 1)
dados['TMO'] = dados['TMO'].where(~dados['TMO'] == 1)
novoHorm = dados['HORMONIO']
novoTmo = dados['TMO']
# print(novoHorm.head(), novoTmo.head())

# 6. Selecionar pacientes com Ano de Diagnóstico até 2019 (ANODIAG ¡= 2019);
ate2019 = dados.where(dados['ANODIAG'] <= 2019)
# print(ate2019.head())

# 7. Retirar pacientes com IDADE menor do que 20 anos;
dados['IDADE']= dados['IDADE'].where(~dados['IDADE'] < 20)
idadeAbaixoD20 = dados['IDADE']
# print(idadeAbaixoD20.head())

# 8. Calcular a diferença em dias entre Diagnóstico e Consulta, Tratamento e Diagnóstico,Tratamento e Consulta, a partir dessas 3 datas (DTCONSULT, DTDIAG e DTTRAT).
dados['DTDIAG'] = pd.to_datetime(dados['DTDIAG'])
dados['DTCONSULT'] = pd.to_datetime(dados['DTCONSULT'])
dados['DTTRAT'] = pd.to_datetime(dados['DTTRAT'])

# Calculando a diferença
dados['CONSDIAG'] = (dados['DTCONSULT'] - dados['DTDIAG']).dt.days
dados['DIAGTRAT'] = (dados['DTTRAT'] - dados['DTDIAG']).dt.days
dados['TRATCONS'] = (dados['DTTRAT'] - dados['DTCONSULT']).dt.days

# Estruturando conforme as regras
dados['CONSDIAG'] = pd.cut(dados['CONSDIAG'],
                         bins=[-float('inf'), 30, 60, float('inf')],
                         labels=[0, 1, 2])

dados['DIAGTRAT'] = pd.cut(dados['DIAGTRAT'].fillna(-1),
                         bins=[-float('inf'), 60, 90, float('inf')],
                         labels=[0, 1, 2],
                         right=False)
dados['DIAGTRAT'] = dados['DIAGTRAT'].cat.add_categories([3])
dados['DIAGTRAT'] = dados['DIAGTRAT'].fillna(3)  # Se DTTRAT é NaN, colocar 3 (não tratou)

# Para a coluna TRATCONS, tratando os casos em que não houve tratamento
dados['TRATCONS'] = pd.cut(dados['TRATCONS'].fillna(-1),
                         bins=[-float('inf'), 60, 90, float('inf')],
                         labels=[0, 1, 2],
                         right=False)
dados['TRATCONS'] = dados['TRATCONS'].cat.add_categories([3])
dados['TRATCONS'] = dados['TRATCONS'].fillna(3)  # Se DTTRAT é NaN, colocar 3 (não tratou)

# print(dados[['CONSDIAG', 'DIAGTRAT', 'TRATCONS']].head())

# 9.Extrair somente o número das colunas DRS e DRSINSTITU;
numColsDrs = dados.where(dados['DRS'].str.extract('(\\d+)'))
numColDrsInsti = dados[dados['DRSINST'].str.extract('(\\d+)')]
#print(numColsDrs.head(), numColDrsInsti.head())

#10. Criar a coluna binária de ́obito, a partir da coluna ULTINFO, onde as categorias 1 e
# 2 representam que o paciente est ́a vivo e as 3 e 4 representam o  ́obito por qualquer
# motivo;
dados['OBITO'] = np.where(dados['ULTINFO'].isin([3, 4]), 1, 0)
obito = dados['OBITO']
# print(obito)

# 11. Retirar colunas
colunas_para_remover = [
    'UFNASC', 'UFRESID', 'CIDADE', 'DTCONSULT', 'CLINICA', 'DTDIAG',
    'BASEDIAG', 'TOPOGRUP', 'DESCTOPO', 'DESCMORFO', 'T', 'N', 'M',
    'PT', 'PN', 'PM', 'S', 'G', 'LOCALTNM', 'IDMITOTIC', 'PSA', 'GLEASON',
    'OUTRACLA', 'META01', 'META02', 'META03', 'META04', 'DTTRAT', 'NAOTRAT',
    'TRATAMENTO', 'TRATHOSP', 'TRATFANTES', 'TRATFAPOS', 'HORMONIO',
    'TMO', 'NENHUMANT', 'CIRURANT', 'RADIOANT', 'QUIMIOANT', 'HORMOANT',
    'TMOANT', 'IMUNOANT', 'OUTROANT', 'HORMOAPOS', 'TMOAPOS', 'DTULTINFO',
    'CICI', 'CICIGRUP', 'CICISUBGRU', 'FAIXAETAR', 'LATERALI', 'INSTORIG',
    'RRAS', 'ERRO', 'DTRECIDIVA', 'RECNENHUM', 'RECLOCAL', 'RECREGIO',
    'RECDIST', 'REC01', 'REC02', 'REC03', 'REC04', 'CIDO',
    'HABILIT', 'HABIT11', 'HABILIT1', 'CIDADEH', 'PERDASEG'
]
#  'DSCCIDO' não existente
dados = dados.drop(columns=colunas_para_remover)
#print(dados.head())